## Drive Hesabına Erişim

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
    print("You are not working in Colab at the moment :(")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/TDDİY'

/content/drive/.shortcut-targets-by-id/18TnDAyZYuScf8Jlx-5YfmnreW4TxwRwf/TDDİY


In [ ]:
!pip install -U gradio
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 K

In [ ]:
import gradio as gr
import pandas as pd
import torch
from transformers import BertForSequenceClassification, AutoTokenizer, AutoModel
import string
import gensim.parsing.preprocessing as gsp
import re
import json

# noktalama işaretlerinin kaldırılması
def remove_punctuation(data):
    data_no_punctuation = [ch for ch in str(data) if ch not in string.punctuation]
    data_no_punctuation = "".join(data_no_punctuation)

    return data_no_punctuation

import nltk
nltk.download('punkt')

# durak kelimelerin kaldırılması
def remove_stopwords(text): 
    stop_words = ['çok', 'at', 'in', 'im', 'acaba','acep','allah','adamakıllı','adeta','ait','altmýþ','altmış','altý'
                  , 'altı','ama','amma','anca','ancak','arada','artýk','aslında','aynen','ayrıca','az','açıkça','açıkçası',
                  'bana','bari','bazen','bazý','bazı','başkası','baţka','belki','ben','benden','beni','benim',
                  'beri','beriki','beþ','beş','beţ','bilcümle','bile','bin','binaen','binaenaleyh','bir','biraz',
                  'birazdan','birbiri','birden','birdenbire','biri','birice','birileri','birisi','birkaç',
                  'birkaçı','birkez','birlikte','birçok','birçoğu','birþey','birþeyi','birşey','birşeyi','birţey',
                  'bitevi','biteviye','bittabi','biz','bizatihi','bizce','bizcileyin','bizden','bize','bizi','bizim',
                  'bizimki','bizzat','boşuna','bu','buna','bunda','bundan','bunlar','bunları','bunların','bunu','bunun',
                  'buracıkta','burada','buradan','burası','böyle','böylece','böylecene','böylelikle','böylemesine',
                  'böylesine','büsbütün','bütün','cuk','cümlesi','da','daha','dahi','dahil','dahilen','daima','dair',
                  'dayanarak','de','defa','dek','demin','demincek','deminden','denli','derakap','derhal','derken',
                  'deđil','değil','değin','diye','diđer','diğer','diğeri','doksan','dokuz','dolayı','dolayısıyla',
                  'doğru','dört','edecek','eden','ederek','edilecek','ediliyor','edilmesi','ediyor','elbet',
                  'elbette','elli','emme','en','enikonu','epey','epeyce','epeyi','esasen','esnasında','etmesi',
                  'etraflı','etraflıca','etti','ettiği','ettiğini','evleviyetle','evvel','evvela','evvelce',
                  'evvelden','evvelemirde','evveli','eđer','eğer','fakat','filanca','gah','gayet','gayetle','gayri',
                  'gayrı','gelgelelim','gene','gerek','gerçi','geçende','geçenlerde','gibi','gibilerden','gibisinden',
                  'gine','göre','gırla','hakeza','halbuki','halen','halihazırda','haliyle','handiyse','hangi','hangisi',
                  'hani','hariç','hasebiyle','hasılı','hatta','hele','hem','henüz','hep','hepsi','her',
                  'herhangi','herkes','herkesin','hiç','hiçbir','hiçbiri','hoş','hulasaten','iken','iki',
                  'ila','ile','ilen','ilgili','ilk','illa','illaki','imdi','indinde','inen','insermi','ise',
                  'ister','itibaren','itibariyle','itibarıyla','iyi','iyice','iyicene','için','iş','işte',
                  'iţte','kadar','kaffesi','kah','kala','kanýmca','karşın','katrilyon','kaynak','kaçı','kelli',
                  'kendi','kendilerine','kendini','kendisi','kendisine','kendisini','kere','kez','keza','kezalik',
                  'keşke','keţke','ki','kim','kimden','kime','kimi','kimisi','kimse','kimsecik','kimsecikler',
                  'külliyen','kýrk','kýsaca','kırk','kısaca','lakin','leh','lütfen','maada','madem','mademki',
                  'mamafih','mebni','međer','meğer','meğerki','meğerse','milyar','milyon','mu','mü','mý','mı',
                  'nasýl','nasıl','nasılsa','nazaran','naşi','ne','neden','nedeniyle','nedenle','nedense',
                  'nerde','nerden','nerdeyse','nere','nerede','nereden','neredeyse','neresi','nereye',
                  'netekim','neye','neyi','neyse','nice','nihayet','nihayetinde','nitekim','niye','niçin',
                  'o','olan','olarak','oldu','olduklarını','oldukça','olduğu','olduğunu','olmadı',
                  'olmadığı','olmak','olması','olmayan','olmaz','olsa','olsun','olup','olur','olursa','oluyor',
                  'on','ona','onca','onculayın','onda','ondan','onlar','onlardan','onlari','onlarýn','onları',
                  'onların','onu','onun','oracık','oracıkta','orada','oradan','oranca','oranla','oraya','otuz',
                  'oysa','oysaki','pek','pekala','peki','pekçe','peyderpey','rağmen','sadece','sahi','sahiden',
                  'sana','sanki','sekiz','seksen','sen','senden','seni','senin','siz','sizden','sizi','sizin',
                  'sonra','sonradan','sonraları','sonunda','tabii','tam','tamam','tamamen','tamamıyla','tarafından',
                  'tek','trilyon','tüm','var','vardı','vasıtasıyla','ve','velev','velhasıl','velhasılıkelam','veya',
                  'veyahut','ya','yahut','yakinen','yakında','yakından','yakınlarda','yalnız','yalnızca','yani',
                  'yapacak','yapmak','yaptı','yaptıkları','yaptığı','yaptığını','yapılan','yapılması','yapıyor',
                  'yedi','yeniden','yenilerde','yerine','yetmiþ','yetmiş','yetmiţ','yine','yirmi','yok','yoksa',
                  'yoluyla','yüz','yüzünden','zarfında','zaten','zati','zira','çabuk','çabukça','çeşitli',
                  'çok','çokları','çoklarınca','çokluk','çoklukla','çokça','çoğu','çoğun','çoğunca','çoğunlukla',
                  'çünkü','öbür','öbürkü','öbürü','önce','önceden','önceleri','öncelikle','öteki','ötekisi','öyle',
                  'öylece','öylelikle','öylemesine','öz','üzere','üç','þey','þeyden','þeyi','þeyler','þu','þuna',
                  'þunda','þundan','þunu','şayet','şey','şeyden','şeyi','şeyler','şu','şuna','şuncacık','şunda',
                  'şundan','şunlar','şunları','şunu','şunun','şura','şuracık','şuracıkta','şurası','şöyle',
                  'ţayet','ţimdi','ţu','ţöyle', 'hala', 'yer', 'güzel', 'büyük']
    stop_words = ['a','acaba','altı','altmış','ama','ancak','arada','artık','asla','aslında','aslında','ayrıca',
                  'az','bana','bazen','bazı','bazıları','belki','ben','benden','beni','benim','beri','beş',
                  'bile','bilhassa','bin','bir','biraz','birçoğu','birçok','biri','birisi','birkaç','birşey',
                  'biz','bizden','bize','bizi','bizim','böyle','böylece','bu','buna','bunda','bundan','bunlar',
                  'bunları','bunların','bunu','bunun','burada','bütün','çoğu','çoğunu','çok','çünkü','da',
                  'daha','dahi','dan','de','defa','değil','diğer','diğeri','diğerleri','diye','doksan','dokuz',
                  'dolayı','dolayısıyla','dört','e','edecek','eden','ederek','edilecek','ediliyor','edilmesi',
                  'ediyor','eğer','elbette','elli','en','etmesi','etti','ettiği','ettiğini','fakat','falan',
                  'filan','gene','gereği','gerek','gibi','göre','hala','halde','halen','hangi','hangisi',
                  'hani','hatta','hem','henüz','hep','hepsi','her','herhangi','herkes','herkese','herkesi',
                  'herkesin','hiç','hiçbir','hiçbiri','i','ı','için','içinde','iki','ile','ilgili','ise',
                  'işte','itibaren','itibariyle','kaç','kadar','karşın','kendi','kendilerine','kendine',
                  'kendini','kendisi','kendisine','kendisini','kez','ki','kim','kime','kimi','kimin',
                  'kimisi','kimse','kırk','madem','mi','mı','milyar','milyon','mu','mü','nasıl','ne',
                  'neden','nedenle','nerde','nerede','nereye','neyse','niçin','nin','nın','niye','nun',
                  'nün','o','öbür','olan','olarak','oldu','olduğu','olduğunu','olduklarını','olmadı',
                  'olmadığı','olmak','olması','olmayan','olmaz','olsa','olsun','olup','olur','olur','olursa',
                  'oluyor','on','ön','ona','önce','ondan','onlar','onlara','onlardan','onları','onların',
                  'onu','onun','orada','öte','ötürü','otuz','öyle','oysa','pek','rağmen','sana','sanki',
                  'sanki','şayet','şekilde','sekiz','seksen','sen','senden','seni','senin','şey','şeyden',
                  'şeye','şeyi','şeyler','şimdi','siz','siz','sizden','sizden','size','sizi','sizi',
                  'sizin','sizin','sonra','şöyle','şu','şuna','şunları','şunu','ta','tabii','tam',
                  'tamam','tamamen','tarafından','trilyon','tüm','tümü','u','ü','üç','un','ün','üzere',
                  'var','vardı','ve','veya','ya','yani','yapacak','yapılan','yapılması','yapıyor','yapmak',
                  'yaptı','yaptığı','yaptığını','yaptıkları','ye','yedi','yerine','yetmiş','yi','yı','yine',
                  'yirmi','yoksa','yu','yüz','zaten','zira','zxtest']
    
    stop_words = ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'bir', 'birkaç', 'birşey', 'biz',
                  'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'den', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem',
                  'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü',
                  'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey',
                  'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'dan']
    word_tokens = nltk.word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    return ' '.join(filtered_text)

# Bazı Türkçe olmayan harflerin türkçe karşılıklarına çevrilmesi
def correct_old_characters(self):
    self = re.sub(r"Â", "A", self)
    self = re.sub(r"Î", "I", self)
    self = re.sub(r"î", "ı", self)
    self = re.sub(r"â", "a", self)
    self = re.sub(r"û", "u", self)
    self = re.sub(r"Û", "U", self)
    return self

# İki karakterden küçük kelimelerin kaldırılması
def remove_two_ch_words(data):
  text = ' '.join([w for w in data.split() if len(w)>2])
  return text

# Kelimelerin küçük harfe çevrilmesi
def lower(self):
    self = re.sub(r"İ", "i", self)
    self = re.sub(r"I", "ı", self)
    self = re.sub(r"Ç", "ç", self)
    self = re.sub(r"Ş", "ş", self)
    self = re.sub(r"Ü", "ü", self)
    self = re.sub(r"Ğ", "ğ", self)
    self = self.lower() # for the rest use default lower
    return self


def auth(username, password):
    if username == "NEU_Kapsul_METEoRS" and password == "4DMNEVO0FCMO3NNK":
        return True
    else:
        return False

def read_csv(csv_file):
  df = pd.read_csv(csv_file, sep = "|")
  return df

def preprocess(df):
  df['remove_punctuation'] = df['text'].apply(remove_punctuation)
  df['remove_stopwords'] = df['remove_punctuation'].apply(remove_stopwords)
  regex_for_numbers = r'([0-9]+)'
  df['remove_numbers'] = df['remove_stopwords'].str.replace(regex_for_numbers, ' ', case=False, regex=True)
  df['correct_chars'] = df['remove_numbers'].apply(lambda x: correct_old_characters(x))
  whitelist = set('abcçdefgğhıijklmnoöpqrsştuüvwxyz ABCÇDEFGĞHIİJKLMNOÖPQRSŞTUÜVWXYZ')

  df['remove_non_tr'] = df['correct_chars'].apply(lambda x: ''.join(filter(whitelist.__contains__, x)))
  # df['remove_2_ch'] = df['remove_non_tr'].apply(remove_two_ch_words)
  df['lower_ch'] = df['remove_non_tr'].apply(lambda x: lower(x))
  df['strip_spaces'] = df['lower_ch'].apply(gsp.strip_multiple_whitespaces)
  # df.dropna(axis=0, inplace=True)

  return df

insult = BertForSequenceClassification.from_pretrained('METEoRS/berturk_model_insult', num_labels=5)
def predict_insult(df, insult):
  model = insult

  tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')

  # CSV dosyasından verileri yükleme
  data = df

  # Tahminleri depolamak için bir dizi oluşturun
  predictions = []

  # Her satır için tahmin yapın
  for index, row in data.iterrows():
      # Cümleyi alın
      sentence = row['strip_spaces']

      # Tokenize edin ve girdiye uygun hale getirin
      inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

      # Modeli kullanarak tahmin yapın
      outputs = model(**inputs)
      probabilities = torch.softmax(outputs.logits, dim=-1)

      # En yüksek olasılığa sahip sınıfın indeksini alın
      predicted_class_index = torch.argmax(probabilities[0]).item()

      # İndeksi etiket adına dönüştürün
      label_dict = {0: "OTHER", 1: "INSULT", 2: "PROFANITY", 3: "SEXIST", 4: "RACIST"}
      predicted_label = label_dict[predicted_class_index]

      # Tahmini diziye ekleyin
      predictions.append(predicted_label)

  return {"INSULT": predictions}

is_offensive=AutoModel.from_pretrained('METEoRS/berturk_model_isoffensive')
def predict_isoffensive(df, is_offensive):
    tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')
    model = is_offensive

    max_length = 64
    # CSV dosyasından verileri yükleme
    data = df

    # Tahminleri depolamak için bir dizi oluşturun
    predictions = []

    # Her satır için tahmin yapın
    for index, row in data.iterrows():
        # Cümleyi alın
        sentence = row['strip_spaces']

        # Tokenize edin ve girdiye uygun hale getirin
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

        # Modeli kullanarak tahmin yapın
        outputs = model(**inputs)
        probabilities = torch.sigmoid(outputs[0])

        # En yüksek olasılığa sahip sınıfın indeksini alın
        predicted_class_index = torch.argmax(probabilities[0]).item()

        # İndeksi etiket adına dönüştürün
        predicted_label = 1 if predicted_class_index == 1 else 0

        # Tahmini diziye ekleyin
        predictions.append(predicted_label)

    # Tahminleri yeni bir sütun olarak CSV dosyasına ekleyin
    
    return {"IS_OFFENSIVE": predictions}

def get_file(file):
    output_file = "output_NEU_Kapsul_METEoRS.csv"
    # For windows users, replace path seperator
    file_name = file.name.replace("\\", "/")
    df = read_csv(file_name)
    df_process = preprocess(df)
    df["INSULT"] = predict_insult(df_process, insult)["INSULT"]
    df["IS_OFFENSIVE"] = predict_isoffensive(df_process, is_offensive)["IS_OFFENSIVE"]
    df.drop(["remove_punctuation", "remove_stopwords", "remove_numbers", "correct_chars", "remove_non_tr", "lower_ch", "strip_spaces"], axis=1, inplace=True)
    df.to_csv(output_file, index=False, sep="|")
    return output_file

input_csv = gr.inputs.File(label="Test dosyasını seçin")
output_csv = gr.outputs.File(label="Tahmin dosyasını indirin")
iface = gr.Interface(get_file, input_csv, output_csv)
# iface = gr.Interface(fn=get_file, inputs=input_csv, outputs=output_csv).launch(share=True)

if __name__ == "__main__":
  iface.queue().launch(share=True, auth=auth, debug=True, inline=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at METEoRS/berturk_model_insult and are newly initialized: ['bert.encoder.layer.332.output.LayerNorm.weight', 'bert.encoder.layer.716.output.LayerNorm.bias', 'bert.encoder.layer.518.attention.self.key.bias', 'bert.encoder.layer.721.attention.self.value.bias', 'bert.encoder.layer.221.attention.self.value.weight', 'bert.encoder.layer.67.output.dense.weight', 'bert.encoder.layer.373.output.LayerNorm.bias', 'bert.encoder.layer.13.attention.self.query.bias', 'bert.encoder.layer.52.intermediate.dense.bias', 'bert.encoder.layer.288.output.LayerNorm.weight', 'bert.encoder.layer.296.attention.self.value.weight', 'bert.encoder.layer.463.attention.self.key.weight', 'bert.encoder.layer.504.intermediate.dense.bias', 'bert.encoder.layer.740.attention.output.LayerNorm.bias', 'bert.encoder.layer.26.attention.output.LayerNorm.weight', 'bert.encoder.layer.222.attention.self.key.bias', 'bert.encoder.layer.360.att

Some weights of the model checkpoint at METEoRS/berturk_model_isoffensive were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at METEoRS/berturk_model_isoffensive and are newly initialized: ['bert.encoder.layer.332.output.LayerNorm.weight', 'bert.encoder.layer.716.output.LayerNorm.bias', 'bert.encoder.layer.518.attention.self.key.bias', 'bert.encoder.layer.721.attention.self.value.bias', 'bert.encoder.layer.221.attention.self.value.weight

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://28f895124d9e853723.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/gradio/routes.py", line 393, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.9/dist-packages/gradio/blocks.py", line 1108, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.9/dist-packages/gradio/blocks.py", line 915, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.9/dist-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<ipython-input-2-a82d3101293f>", line 235, in get_file
    file_name = file.name.replace("\\", "/")
AttributeError: 'NoneTyp